# show the max and min in each emotion

In [ ]:
import os
import pandas as pd

emolist = 'anger, disgust, fear, happiness, sadness, surprise'.split(', ')
result = {x:[] for x in emolist}
myfolder = 'image_analysis/Exp_collections/230210Exp13Lighting_sys_500/'

for CURR_EMO in emolist:
#     CURR_EMO = 'anger'
    images = [x for x in os.listdir(os.path.join(myfolder, CURR_EMO)) if 'png' in x]

    for img in images:
        test = os.path.join(myfolder, CURR_EMO, img)
        csvname = test.split('.png')[0] + '_emotion.csv'
        tmp = pd.read_csv(csvname)
        result[CURR_EMO].append(float(tmp[CURR_EMO]))

    print(CURR_EMO)
    print(f'max: {max(result[CURR_EMO])}')
    print(f'min: {min(result[CURR_EMO])}')

In [ ]:
# plot for each emotions

In [ ]:
result

In [ ]:
float(tmp[CURR_EMO])

# Model range 

In [ ]:
from intensityNet import * 
global intensityModel
intensityModel = ''

def intensityNet_analysis(img, target_emotion, is_save_csv=True):
    # remember to set it before doing analysis

    global intensityModel
    # use intensityModel to detect emo
    detection_res = intensityModel.detect_emo(Image.open(img))
    detection_res = detection_res.tolist()
    output = detection_res[get_target(target_emotion)]

#     # create a pd dataframe
#     detection_res = pd.DataFrame([detection_res], columns=["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"])
#     # Create DataFrame

#     if is_save_csv:
#         csv_emotion_name = img[:-4]+"_intensitynet.csv"
#         detection_res.to_csv(csv_emotion_name)

    # result = tmp_res[target_emotion]
    return output

def setIntensityModel(target_emotion):
    global intensityModel
    # Load the model
    # ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
    print("target_emotion:", target_emotion)
    if target_emotion.lower() in ["anger", 'angry']:
        model_path = "new_models/angry_fold3_epoch7.pt"
    elif target_emotion.lower() in ["disgust"]:
        model_path = "new_models/disgust_fold3_epoch7.pt"
    elif target_emotion.lower() in ["fear"]:
        model_path = "new_models/fear_fold3_epoch6.pt"
    elif target_emotion.lower() in ["happiness", "happy"]:
        model_path = "new_models/happy_fold2_epoch7.pt"
    elif target_emotion.lower() in ["sadness", "sad"]:
        model_path = "new_models/sad_fold2_epoch6.pt"
    elif target_emotion.lower() in ["surprise"]:
        model_path = "new_models/surprise_fold3_epoch5.pt"
    else:
        model_path = ""

    intensityModel = IntensityNet_type1(model_path)
    return 1

# intensityNet_analysis()


In [ ]:
emolist = 'anger, disgust, fear, happiness, sadness, surprise'.split(', ')
result = {x:[] for x in emolist}
myfolder = 'image_analysis/Exp_collections/230210Exp13Lighting_sys_500/'


# CURR_EMO = 'anger'

global intensityModel
intensityModel = ''



for CURR_EMO in emolist:
    setIntensityModel(CURR_EMO)
    images = [x for x in os.listdir(os.path.join(myfolder, CURR_EMO)) if 'png' in x]
    
    for i in images:
        result[CURR_EMO].append(intensityNet_analysis(os.path.join(myfolder, CURR_EMO, images[0]), CURR_EMO))
        
    print(CURR_EMO)
    print(len(result[CURR_EMO]))
    print(f'max: {max(result[CURR_EMO])}')
    print(f'min: {min(result[CURR_EMO])}')

    

In [ ]:
# Test the mixed model

# Function to calculate the mixed output with nonlinear transition using a sigmoid function
def calculate_output_nonlinear(A, B, threshold=0.75, alpha=0.8, B_min=0.39, B_max=0.64, output_min=0.75, output_max=1.2, k=10):
    """
    Calculate the mixed output based on values of A and B, with a threshold-based decision rule.
    
    Parameters:
    A (float): Value from distribution A.
    B (float): Value from distribution B.
    threshold (float): Threshold for A to determine when to mix. Default is 0.75.
    alpha (float): Weighting factor for B' in the mix when A > threshold. Default is 0.8.
    B_min (float): Minimum value of B's original range. Default is 0.39.
    B_max (float): Maximum value of B's original range. Default is 0.64.
    output_min (float): Minimum value of the target range for B' (after scaling). Default is 0.75.
    output_max (float): Maximum value of the target range for B' (after scaling). Default is 1.2.

    Returns:
    float: Final output value based on A and B.
    """
        
    # If A is below or equal to the threshold, output A directly
    if A <= threshold:
        return A
    
    # Scale B to fit within the desired range [output_min, output_max]
    B_mapped = output_min + (B - B_min) * (output_max - output_min) / (B_max - B_min)
    
    # Apply a sigmoid-based weight for smooth transition
    weight = 1 / (1 + np.exp(-k * (A - threshold)))  # Sigmoid function for smoother blending
    
    # Calculate the smooth nonlinear mixed output
    output = weight * (alpha * B_mapped + (1 - alpha) * A) + (1 - weight) * A
    
    return output





## Test the mixed model on old dataset

# extract probe from past experiment

In [ ]:
import numpy as np 
def omega(x):
    return 1.0 / (1 + np.exp(-k * ( x - threshold))

# difference between pyfeat value and resmasknet value (DONE)

In [ ]:
# pyfeat value
from feat import Detector
import os
import cv2
import numpy as np
# from SiameseRankNet import *
from resmasknet import ResMaskNet
from torchvision.transforms import transforms

detector = Detector(emotion_model = "resmasknet", landmark_model='mobilefacenet')

def get_face_box(start_x, start_y, end_x, end_y):
    center_x, center_y = (start_x + end_x) // 2, (start_y + end_y) // 2
    square_length = ((end_x - start_x) + (end_y - start_y)) // 2 // 2
    square_length *= 1.1
    start_x = int(center_x - square_length)
    start_y = int(center_y - square_length)
    end_x = int(center_x + square_length)
    end_y = int(center_y + square_length)
    return start_x, start_y, end_x, end_y


result = {}
myfolder = 'image_analysis/'
CURR_EMO = 'anger'
images = [x for x in os.listdir(os.path.join(myfolder, CURR_EMO)) if 'png' in x]
image_size = (224, 224)
transform = transforms.Compose(
            [transforms.ToPILImage(), transforms.ToTensor()]
        )
model = ResMaskNet()
for i in images[:]:
    print(i)
    test = os.path.join(myfolder, CURR_EMO, i)
    res = detector.detect_image(test)
    facebox = res.iloc[:,1:5]
    emotions = res.iloc[:, -8:]
    print(f'use pyfeat:\n{[round(x, 5) for x in np.array(emotions.iloc[:, :]).tolist()[0][:-1]]}')
    
    frame = cv2.imread(test)
    face = detector.detect_faces(frame)[0][:4]
    emotion = model.detect_emo(frame=frame, detected_face=[face])
    print(emotion)
    print(f'use model directly:\n{list(round(x, 5) for x in emotion[0].tolist())}')
    
    test = os.path.join(myfolder, CURR_EMO, i)
    res = detector.detect_image(test)
    facebox = res.iloc[:,1:5]
    emotions = res.iloc[:, -8:]
    print(f'use pyfeat again:\n{[round(x, 5) for x in np.array(emotions.iloc[:, :]).tolist()[0][:-1]]}')
    
    print()
    

In [ ]:
face

In [ ]:
import pandas as pd
new_df = pd.DataFrame(emotion, columns=["anger", "disgust", "fear", "happiness", "sadness", "surprise", "neutral"])
print(new_df)

In [ ]:
import pandas as pd
from feat import Detector
import os
import cv2

def get_target(emotion_name):
    # Anger, Disgust, Fear, Happiness, Sadness, Surprise, Neutral
    # or lowercase
    if emotion_name in ["Anger", "anger"]:
        return 0
    elif emotion_name in ["Disgust", "disgust"]:
        return 1
    elif emotion_name in ["Fear", "fear"]:
        return 2
    elif emotion_name in ["Happiness", "happiness"]:
        return 3
    elif emotion_name in ["Sadness", "sadness"]:
        return 4
    elif emotion_name in ["Surprise", "surprise"]:
        return 5
    elif emotion_name in ["Neutral", "neutral"]:
        return 6
image_name = 'image_analysis/fear/2024_11_14_17_14_20_fear_0.png'
target_emotion = 'disgust'

facebox = detector.detect_faces(cv2.imread(image_name))[0]
rmn_model = ResMaskNet()
targetID = get_target(target_emotion)
print(image_name)
rmn_res = rmn_model.detect_emo(frame=cv2.imread(image_name), detected_face=[facebox])
new_df = pd.DataFrame(rmn_res, columns=["anger", "disgust", "fear", "happiness", "sadness", "surprise", "neutral"])
new_df['input'] = image_name


# -----------
# old version
# -----------
image_prediction = detector.detect_image(image_name)
df = image_prediction.head()

targetID = get_target(target_emotion)

assert list(new_df[target_emotion])[0] == list(df[target_emotion])[0], "The result is not the same, old: {}, new: {}".format(list(df[target_emotion])[0], list(new_df[target_emotion])[0])
# return emo_df.iloc[0,targetID]
print(new_df[target_emotion])
print(df[target_emotion])

In [ ]:
pprint(detector.detect_emotions)

# test bayesian optimization using probe

In [ ]:
from bayes_opt import BayesianOptimization

def generate_pbounds(axes, target_emotion):
        pbounds_dic = {}
        for i in axes:
            if target_emotion == 'happiness' and i in [0, 1]:
                # TODO need to verify
                happy_upper = 120
                pbounds_dic['x{}'.format(i)] = (0, happy_upper)
            # elif i in [0, 1]:
            #     pbounds_dic['x{}'.format(i)] = (0, 160)
            # else:
            pbounds_dic['x{}'.format(i)] = (0, 255)

        # mouth
        pbounds_dic['x32'] = (0, 110)

        # dangerous point
        pbounds_dic['x8'] = (-255, 255)
        pbounds_dic['x18'] = (-255, 255)

        print(pbounds_dic)
        return pbounds_dic
    
target_emotion = 'target_emotion'
all_axes_for_emotions = [1, 6, 8, 10, 11, 16, 18, 20, 28, 29, 30, 32]
pbounds = generate_pbounds(all_axes_for_emotions, target_emotion)

def middle_function(**kwargs):
    print(kwargs)
    return 'haha'


optimizer = BayesianOptimization(
        f=middle_function,
        # Define HyperParameter Space
        # acquisition_function=acq,
        pbounds = pbounds,
        random_state=486,
        verbose=2)

In [ ]:
from bayes_opt import UtilityFunction

ucb = UtilityFunction(kind='ucb',
                          kappa=2.576,
                          xi=0.0,
                          kappa_decay=1,
                          kappa_decay_delay=0)
suggest = optimizer.suggest(ucb)
def check_suggestion(suggestion):
    # x1 and x6
    print('checked')
    if suggestion['x1'] + suggestion['x6'] < 420:
        # if the upper lid and lower lid are too close to each other, we should not return 0
        print("[INFO]Eye closing Constraints, search another point")
        return 0
    return 1

print(suggest)
while not check_suggestion(suggest):
    suggest = optimizer.suggest(ucb)
if check_suggestion(suggest):
    print('ok')



In [ ]:
k = optimizer.probe(ucb)
print(k)

# get prototype probe number

In [1]:
def get_default_pose(param, pos):
    res = {}
    for p in pos:
        res[f'x{p}'] = param[p - 1]
        if p + 1 in [9, 13, 19, 23] and param[p - 1] == 0 and param[p] != 0:
            print('hi')
            print(f'x{p}')
            print(f'x{p+1} {param[p]}')
            res[f'x{p}'] = - param[p]
            
        # [8, 12, 18, 22]. If p > 0, we do nothing. If p < 0, [8, 12, 18, 22] = 0, [9, 13, 19, 23] = -p
    
    return res

import defaultPose

for exp in ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']:
    target = defaultPose.prototypeFacialExpressions[exp]
    print(exp, target)
    res = get_default_pose(target, [1, 6, 8, 10, 11, 16, 18, 20, 28, 29, 30, 32])
    probe = ""
    for k, v in res.items():
        probe += f"'{k}':{v}, "
    print('{' + probe[:-2] + '}')
    

anger [0, 0, 128, 128, 128, 255, 255, 0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 128, 122]
{'x1':0, 'x6':255, 'x8':0, 'x10':0, 'x11':255, 'x16':0, 'x18':0, 'x20':0, 'x28':0, 'x29':0, 'x30':0, 'x32':0}
disgust [86, 86, 128, 128, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 0, 0, 128, 128, 122]
{'x1':86, 'x6':0, 'x8':0, 'x10':0, 'x11':0, 'x16':0, 'x18':0, 'x20':0, 'x28':0, 'x29':0, 'x30':255, 'x32':0}
fear [0, 0, 128, 128, 128, 0, 0, 255, 0, 255, 255, 255, 0, 255, 255, 0, 0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 0, 0, 0, 0, 0, 128, 128, 122]
{'x1':0, 'x6':0, 'x8':255, 'x10':255, 'x11':255, 'x16':0, 'x18':0, 'x20':255, 'x28':0, 'x29':0, 'x30':0, 'x32':0}
happiness [86, 86, 128, 128, 128, 255, 255, 0, 255, 0, 0, 0, 255, 0, 0, 255, 255, 255, 0, 0, 0, 255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 128, 122]
hi
x8
x9 255
{'x1':86, 'x6':255, 'x8':-255, 'x10':0, 'x11':0, 'x16':255, 'x18':255, 'x20':0, 'x28':0, 'x29':0, 'x30':0, 

# get human average probe

In [2]:
import human_data
import os

hd = human_data.average_human_order
# print(hd)
emo = 'surprise'
emoo = emo
# print(hd[emo][:10])

tr = [emo + '_' + x.split('_')[1] for x in hd[emoo][:10]]
print(tr)
tr = [t.split('.png')[0] + '_axes_data.csv' for t in tr]
print(tr)

basefolder = 'image_analysis/Exp_collections/230210Exp13Lighting_sys_500/'
files = os.listdir(os.path.join(basefolder, emo))

def get_csv_name():
    res = []
    
    for t in tr:
        for file in files:
            if t in file:
                res.append(file)
                continue

    return res

csvname = get_csv_name()
print(csvname)

['surprise_275.png', 'surprise_245.png', 'surprise_261.png', 'surprise_369.png', 'surprise_337.png', 'surprise_121.png', 'surprise_428.png', 'surprise_407.png', 'surprise_440.png', 'surprise_106.png']
['surprise_275_axes_data.csv', 'surprise_245_axes_data.csv', 'surprise_261_axes_data.csv', 'surprise_369_axes_data.csv', 'surprise_337_axes_data.csv', 'surprise_121_axes_data.csv', 'surprise_428_axes_data.csv', 'surprise_407_axes_data.csv', 'surprise_440_axes_data.csv', 'surprise_106_axes_data.csv']
['2023_02_10_18_49_17_surprise_275_axes_data.csv', '2023_02_10_18_48_09_surprise_245_axes_data.csv', '2023_02_10_18_48_46_surprise_261_axes_data.csv', '2023_02_10_18_52_52_surprise_369_axes_data.csv', '2023_02_10_18_51_36_surprise_337_axes_data.csv', '2023_02_10_18_43_32_surprise_121_axes_data.csv', '2023_02_10_18_55_17_surprise_428_axes_data.csv', '2023_02_10_18_54_24_surprise_407_axes_data.csv', '2023_02_10_18_55_47_surprise_440_axes_data.csv', '2023_02_10_18_43_00_surprise_106_axes_data.csv

In [50]:
import pandas as pd
import re

for c in csvname:
    tmp = pd.read_csv(os.path.join(basefolder, emo, c))
    tmp = tmp.to_dict()
    for k, v in tmp.items():
        if 'x' in k:
            tmp[k] = round(v[0])
            
    match = re.search(r".*(surprise_\d+_axes_data\.csv)", c)
    tmp[emo] = match.group(1)
    print()
    print(tmp)
    

{'surprise': 'surprise_275_axes_data.csv', 'x1': 0, 'x10': 0, 'x11': 0, 'x16': 0, 'x18': 255, 'x19': 0, 'x20': 0, 'x28': 255, 'x29': 0, 'x30': 0, 'x32': 255, 'x6': 0, 'x8': 255, 'x9': 0}
{'surprise': 'surprise_245_axes_data.csv', 'x1': 0, 'x10': 0, 'x11': 0, 'x16': 0, 'x18': 0, 'x19': 0, 'x20': 255, 'x28': 0, 'x29': 149, 'x30': 0, 'x32': 255, 'x6': 0, 'x8': 255, 'x9': 21}
{'surprise': 'surprise_261_axes_data.csv', 'x1': 0, 'x10': 255, 'x11': 0, 'x16': 0, 'x18': 255, 'x19': 0, 'x20': 255, 'x28': 72, 'x29': 0, 'x30': 0, 'x32': 255, 'x6': 0, 'x8': 255, 'x9': 0}
{'surprise': 'surprise_369_axes_data.csv', 'x1': 0, 'x10': 255, 'x11': 0, 'x16': 255, 'x18': 0, 'x19': 0, 'x20': 255, 'x28': 0, 'x29': 0, 'x30': 0, 'x32': 255, 'x6': 140, 'x8': 255, 'x9': 255}
{'surprise': 'surprise_337_axes_data.csv', 'x1': 0, 'x10': 255, 'x11': 0, 'x16': 0, 'x18': 9, 'x19': 0, 'x20': 0, 'x28': 0, 'x29': 255, 'x30': 0, 'x32': 255, 'x6': 0, 'x8': 255, 'x9': 149}
{'surprise': 'surprise_121_axes_data.csv', 'x1': 0, '

# get feat order probe as baseline

In [16]:
import human_data
import os

hd = human_data.feat_order
# print(hd)
emoo = 'angry'
if emoo == 'angry':
    emo = 'anger'
elif emoo == 'happy':
    emo = 'happiness'
elif emoo == 'sad':
    emo = 'sadness'
else:
    emo = emoo

# print(hd[emo][:10])

tr = [emo + '_' + x.split('_')[1] for x in hd[emoo][:10]]
# print(tr)
tr = [t.split('.png')[0] + '_axes_data.csv' for t in tr]
print(tr)

basefolder = 'image_analysis/Exp_collections/230210Exp13Lighting_sys_500/'
files = os.listdir(os.path.join(basefolder, emo))

def get_csv_name():
    res = []
    for t in tr:
        for file in files:
            if t in file:
                res.append(file)
                continue
    return res

csvname = get_csv_name()
print(csvname)

import pandas as pd
import re
for c in csvname:
    tmp = pd.read_csv(os.path.join(basefolder, emo, c))
    tmp = tmp.to_dict()
#     print(tmp)
    for k, v in tmp.items():
        if 'x' in k:
            tmp[k] = round(v[0])
            
    match = re.search(r".*((anger|disgust|fear|happiness|sadness|surprise)_\d+_axes_data\.csv)", c)
    tmp[emo] = match.group(1)
    print(tmp[emo])
    print({k:v for k, v in tmp.items() if 'x' in k})
    print()
    